In [6]:
import websockets

WS_EOS = '//EOS//'

In [7]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
BOS, EOS = "<s>", "</s>"

# fmt: off
DEFAULT_SYSTEM_PROMPT = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""
# fmt: on

from typing import Literal, TypedDict, List

Role = Literal["system", "user", "assistant"]

class Message(TypedDict):
    role: Role
    content: str
    
Dialog = List[Message]

dialog: Dialog = [
    {"role": "system","content": DEFAULT_SYSTEM_PROMPT,},
    {"role": "user", "content": "What is love ?"},
]
if dialog[0]["role"] == "system":
    dialog = [
        {
            "role": dialog[1]["role"],
            "content": B_SYS + dialog[0]["content"] + E_SYS + dialog[1]["content"],
        }
    ] + dialog[2:]


In [8]:
def get_tokens(dialog: Dialog) -> str:
    dialog_tokens = sum(
        [
            [f"{BOS}{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()}{EOS}"]
            for prompt, answer in zip(dialog[::2], dialog[1::2],)
        ],
        [],
    )
    dialog_tokens += f"{BOS}{B_INST} {(dialog[-1]['content']).strip()} {E_INST}{EOS}",
    return ''.join(dialog_tokens)

In [9]:
async def send_message():
    uri = "ws://localhost:8765"  # WebSocket server address

    async with websockets.connect(uri) as websocket:
        await websocket.send(get_tokens(dialog))

        while True:
            response = await websocket.recv()
            if response == WS_EOS:
                break
            else:
                print(response, end="", flush=True)

In [10]:
await send_message()

 Hello! I'm here to help you with your question. Love is a complex and multifaceted emotion that can be difficult to define. At its core, love refers to a deep sense of affection, attachment, or connection towards another person or entity. It can take many forms, including romantic love, familial love, platonic love, and self-love.
Love can be characterized by feelings of warmth, tenderness, compassion, and commitment. It involves caring for someone else's well-being and desiring their happiness, even if it means making sacrifices for their benefit. Love can also involve a sense of mutuality, where both parties feel valued and appreciated by each other.
However, it's important to note that love is not always easy or straightforward. It can be challenging, messy, and complex, especially in long-term relationships. Additionally, different people may have different definitions and experiences of love, which can make it difficult to pin down a single, universally accepted definition.
Ultim